In [1]:
import pandas as pd
import pyodbc
import warnings
import google.generativeai as genai
import time
from tqdm import tqdm

ModuleNotFoundError: No module named 'google.generativeai'

### Conexion a base de datos e ia


In [ ]:
# Necesitaras crear un archivo GEMINI_API_KEYS.txt con las claves de Gemini que tengas
with open('../GEMINI_API_KEY.txt', 'r') as f:
    # Importante poner .strip() para quitar posibles '\n'
    genai.configure(api_key=f.readline().strip())
with open('../Estructura_Base_de_Datos_Gemini.txt', 'r') as f:
    estructura_sql = ''.join(f.readlines())

model = genai.GenerativeModel('gemini-2.0-flash')

# Conectarse a Azure SQL
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=sql-server-mamba.database.windows.net;'
    'DATABASE=sql-db-mamba;'
    'UID=sql-admin;'
    'PWD=serverpwd1!'
)

### Estructura de la base de datos

In [3]:
print(estructura_sql)

CREATE TABLE [dbo].[Taxonomia]( -- Tabla que define las especies (taxones)
	[taxonid] [int] NOT NULL, -- Identificador
	[name] [varchar(MAX)] NULL, -- Nombre científico
	[taxonrank] [varchar(MAX)] NULL, -- Rango del taxón, puede ser: "Species", "Subspecies", "Variety", "Form"
	[origen] [varchar(MAX)] NULL, -- Si es especie nativa o no
	[taxonomicgroup] [varchar(MAX)] NULL, -- El grupo taxonómico
	[kingdom] [varchar(MAX)] NULL, -- El reino taxonómico
	[phylum] [varchar(MAX)] NULL, -- El philo taxonómico
	[class] [varchar(MAX)] NULL, -- La clase taxonómico
	[taxonorder] [varchar(MAX)] NULL, -- El orden taxonómico
	[family] [varchar(MAX)] NULL, -- La familia taxonómica
	[genus] [varchar(MAX)] NULL, -- El género taxonómico
	[subgenus] [varchar(MAX)] NULL, -- El subgénero taxonómico
	[nametype] [varchar(MAX)] NULL -- Tipo de nombre, puede ser "Aceptado" o "Sinónimo"
)

CREATE TABLE [dbo].[NombresEspecies]( -- Tabla de nombres para cada taxón
	[idtaxon] [int] NULL, -- Identificador
	[nombre_

ejemplo base de datos

In [4]:
df = pd.read_excel('PROMPTS.xlsx')
df.sample(n=5, random_state=0)

,idtaxon,Prompt,Texto usado,Tablas usadas,Columnas usadas,Nueva descripción,Prompt alternativo 1,Prompt alternativo 2,Descripción alternativa 1,Descripción alternativa 2
8,14085,¿Me puedes indicar si puedo ver estas especies...,PR-Z 180. Novillas-Magallón-Bulbuente | Zarago...,CuadriculasRutas | CuadriculasEspecies | Rutas,ID_Ruta | CUADRICULA | idtaxon | cuadricula | ...,El fumarel cariblanco es un ave de tamaño simi...,Quiero saber si puedo encontrar estas especies...,Dame más rutas donde pueda ver a estas especies,"El fumarel común se parece a los charranes, pe...",El fumarel común se distingue por su vuelo tra...
50,11301,Asocia las especies a pokemon ya existentes y ...,Reptiles,Taxonomia,taxonid | taxonomicgroup,Esta especie recuerda al pokemon Serperior por...,Mi hijo y yo hemos estado jugando a Pokemon re...,Encuentra un Pokémon que se asemeje a cada una...,Esta especie comparte con el pokemon Serperior...,"Con un cuerpo largo, sin patas y cubierto de e..."
43,12089,Haz las descripciones más técnicas e incluye i...,Abejorros medianos (longitud: 17-20 mm en rein...,DescripcionesEspecies | Taxonomia,idtaxon | Descripcion | taxonid | genus,Los abejorros del género Bombus son de tamaño ...,"Haz descripciones más formales y detalladas, i...",Utiliza terminología científica y asegúrate de...,El género Bombus se caracteriza por su tamaño ...,Los abejorro del género Bombus muestran un pat...
24,11198,Incluye el filo al que pertenece cada especie,Chordata,Taxonomia,taxonid | phylum,Filo - Chordata. El halcón de Eleonora es un h...,Indica el phylum al inicio de cada descripción...,"Al principio de cada descripción, incluye una ...",Filo - Chordata. El halcón de Eleonora es un h...,Filo - Chordata. El halcón de Eleonora es un a...
63,10998,Dime si tiene otros nombres comunes. Si no los...,Chub | Inglés,NombresEspecies,idtaxon | espreferente | nombre_comun | idioma,"Es un pez grande, de la familia de las carpas,...","Si la especie tiene otros nombres comunes, añá...",Incluye otros nombres comunes si existen.,"Es un pez grande, de hasta 60 cm, con cuerpo a...","Es un pez grande, de cuerpo alargado y platead..."


### Prompt para eviar a gemenini

In [5]:
PROMPT_GEMINI = """
Esta es la definción de mi base de datos en SQL:

{estructura_sql}

Tengo una aplicación cuyo objetivo es describir especies en el contexto de rutas, de forma que el usuario puede personalizar las descripciones.
El usuario ha metido el siguiente texto:

"{prompt_usuario}"

Tu objetivo es crear una consulta SQL de forma que, dado un idtaxon/taxonid, ejecutándola se extraiga toda la información relevante para poder la petición.
Puedes asumir que tienes una descripción de la cual partir, un idtaxon/taxonid, el nombre aceptado de la especie y el nombre común preferente.
Si necesitas usar esta información (por ejemplo, si necesitas la taxonomía de la especie), utiliza <idtaxon>, <nombre_aceptado> y <nombre_preferente>.
Asegúrate de que la consulta SQL solo contiene las columnas e información relevante.
Si el usuario te ha pedido algo que no es relevante para las descripciones o hace referencia a imágenes u algo que no te he mencionado, devuelve la consulta:
SELECT Null
Además, si el usuario pide "otras" provincias, rutas o clases, haz caso omiso de la palabra "otras", "otros" o similar.
Razona primero qué tablas e información necesitas, y devuelve la consulta al final.
La consulta debe estar completa a falta de posiblemente <idtaxon>, <nombre_aceptado> y <nombre_preferente>.
"""

 ### Prompts principales y sus equivalentes alternativos

In [6]:
index_dict = {}
for i, row in df.iterrows():
    index_dict[row['Prompt']] = i
    index_dict[row['Prompt alternativo 1']] = i
    index_dict[row['Prompt alternativo 2']] = i

In [7]:
df['Indice'] = df['Prompt'].apply(lambda x: index_dict[x])

In [ ]:
# Hacer dataframe indice -> Consulta
prompts_df = pd.concat(
    [
        df[['Indice', 'Prompt']],
        df[['Indice', 'Prompt alternativo 1']].rename(columns={'Prompt alternativo 1': 'Prompt'}),
        df[['Indice', 'Prompt alternativo 2']].rename(columns={'Prompt alternativo 2': 'Prompt'})
    ]
).drop_duplicates().sort_values(by='Indice')
prompts_df['SQL'] = ''
prompts_df.head()

,Indice,Prompt,SQL
0,2,Dime en qué otras provincias puedo encontrar e...,
0,2,"¿En qué lugares de España, a nivel provincial,...",
0,2,Hazme una lista de provincias adicionales dond...,
3,5,Me gustaría que me dijeras si estas especies a...,
3,5,¿Puedes proporcionar datos sobre su presencia ...,


### Agrupación de Prompts Semánticamente Equivalentes y Preparación para Generación de SQL

In [ ]:
warnings.filterwarnings('ignore')

nombre_aceptado = "Artemisia thuscula Cav."
nombre_preferente = "Abrotano"
idtaxon = 61
gemini_calls = 0

for index in tqdm(set(prompts_df['Indice'])):
    prompts = prompts_df[prompts_df['Indice'] == index]
    if prompts.iloc[0]['SQL'] != '':
        continue
    call_gemini = True
    prompt_number = 0
    while call_gemini:
        if prompt_number >= len(prompts):
            print(f"Prompt '{prompt}', con indice {index} ha generado un error, la consulta devuelta ha sido:\n{answers[0]}\n")
            call_gemini = False
            continue
        prompt = prompts.iloc[prompt_number]['Prompt']
        messages = [
            {'role': 'user',
            'parts': [PROMPT_GEMINI.format(estructura_sql=estructura_sql, prompt_usuario=prompt)]}
        ]
        prompt_number += 1
        gemini_calls += 1
        if gemini_calls % 15 == 0:
            time.sleep(60)
        response = model.generate_content(messages)
        answers = [c.content.parts[0].text.strip() for c in response.candidates]
        answers = [a.split("```sql")[-1].split("```")[0] for a in answers]
        try:
            _ = pd.read_sql(answers[0].replace("<idtaxon>", str(idtaxon)).replace("<nombre_aceptado>", nombre_aceptado).replace("<nombre_preferente>", nombre_preferente), conn)
            prompts_df.loc[prompts.index, 'SQL'] = answers[0].strip()
            call_gemini = False
        except Exception as e:
            prompt_number += 1

warnings.filterwarnings('default')
prompts_df.to_excel('../Text2SQL_Data.xlsx', index=False)

100%|██████████| 51/51 [02:01<00:00,  2.37s/it]
